# IMDB 영화평감성분석

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../00.data/IMDB/labeledTrainData.tsv',sep='\t',header=0,quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [4]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [5]:
# <br /> 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />', ' ')

In [6]:
# 영어 이외의 문자는 공백으로 변환
import re
df['review'] = df.review.apply(lambda x:re.sub('[^a-zA-Z]', ' ', x))

In [7]:
from sklearn.model_selection import train_test_split

feature_df = df.drop(['id', 'sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(
    feature_df, df.sentiment, test_size=0.3, random_state=156
)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

- CountVectorizer

In [9]:
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2))
count_vect.fit(X_train.review)
X_train_count = count_vect.transform(X_train.review)
X_test_count = count_vect.transform(X_test.review)

In [10]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_count, y_train)
pred = lr_clf.predict(X_test_count)
accuracy_score(y_test, pred)

0.886

- TfidfVectorizer

In [11]:
tfidf_vect = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
tfidf_vect.fit(X_train.review)
X_train_tfidf = tfidf_vect.transform(X_train.review)
X_test_tfidf = tfidf_vect.transform(X_test.review)

In [12]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_tfidf, y_train)
pred = lr_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred)

0.8936

### 모델 저장하고 불러오기

In [13]:
import joblib
# 모델 저장하기
joblib.dump(tfidf_vect, 'model/imdb_vect.pkl')
joblib.dump(lr_clf, 'model/imdb_lr.pkl')

['model/imdb_lr.pkl']

In [14]:
# 모델 불러오기
new_vect = joblib.load('model/imdb_vect.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

In [15]:
new_X_test = new_vect.transform(X_test.review)

In [16]:
pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

0.8936

### Pipeline을 써서 학습/예측/평가

In [17]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + Logisitic Regression 정확도: {acc:.4f}')

Count Vectorizer + Logisitic Regression 정확도: 0.8860


In [18]:
joblib.dump(pipeline, 'model/pipeline.pkl')

['model/pipeline.pkl']

In [19]:
new_pipe = joblib.load('model/pipeline.pkl')

In [20]:
pred = new_pipe.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + Logisitic Regression 정확도: {acc:.4f}')

Count Vectorizer + Logisitic Regression 정확도: 0.8860


In [21]:
count_vect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 2),
 'preprocessor': None,
 'stop_words': 'english',
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [22]:
tfidf_vect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 2),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [24]:
params = {
    'count_vect__ngram_range' : [(1,1),(1,2)],
    'count_vect__max_df': [100,300, 700],
    'lr_clf__C': [1,10]
}

In [26]:
from sklearn.model_selection import GridSearchCV

grid_pipe = GridSearchCV(new_pipe, param_grid=params, cv=3, scoring='accuracy',verbose=1,n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_,grid_pipe.best_score_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 28.6min
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed: 42.7min finished
{'count_vect__max_df': 700, 'count_vect__ngram_range': (1, 2), 'lr_clf__C': 1} 0.8736571714181699


In [29]:
best = grid_pipe.best_estimator_
pred = best.predict(X_test.review)
accuracy_score(y_test, pred)

0.8794666666666666

In [30]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf',LogisticRegression())
])
params = {
    'tfidf_vect__ngram_range' : [(1,1),(1,2)],
    'tfidf_vect__max_df': [100,300, 700],
    'lr_clf__C': [1,10]
}
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy',verbose=1,n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_,grid_pipe.best_score_)
best = grid_pipe.best_estimator_
pred = best.predict(X_test.review)
print(accuracy_score(y_test, pred))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 33.3min finished
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.8822287469759523
0.8872
